In [1]:
!git clone https://github.com/feralvam/easse.git
%cd easse
!pip install -e .
%cd ..

Cloning into 'easse'...
remote: Enumerating objects: 1937, done.
remote: Counting objects: 100% (504/504), done.
remote: Compressing objects: 100% (303/303), done.
remote: Total 1937 (delta 289), reused 347 (delta 194), pack-reused 1433
Receiving objects: 100% (1937/1937), 33.79 MiB | 19.87 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.
/content/easse
Obtaining file:///content/easse
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git to /tmp/pip-install-d13a2nk4/tseval_ab9e931ca948471cb2c7ad496f7e0af6
  Running command git clone -q https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-d13a2nk4/tseval_ab9e931ca948471cb2c7ad496f7e0af6
  Cloning https://github.com/cisnlp/simalign.git to /tmp/pip-install-d13a2nk4/simalign_a2c5edb65f604044b41b75ca3edac4ad
  Running command git clone -q https://github.com/cisnlp/simalign.git /tmp/pip-install-d13a2nk4/simalign_a2c5edb65f604044b41b75ca3edac4ad
  Installing build dependencie

In [2]:
!git clone https://github.com/facebookresearch/muss.git
%cd muss/
!pip install -e .   # Install package
!python -m spacy download en_core_web_md
%cd ..

Cloning into 'muss'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (395/395), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 395 (delta 188), reused 354 (delta 158), pack-reused 0
Receiving objects: 100% (395/395), 5.38 MiB | 18.05 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/muss
Obtaining file:///content/muss
  Cloning git://github.com/feralvam/easse.git to /tmp/pip-install-6pgglpio/easse_3312aeae90324a679844328251ed3d9b
  Running command git clone -q git://github.com/feralvam/easse.git /tmp/pip-install-6pgglpio/easse_3312aeae90324a679844328251ed3d9b
  Cloning git://github.com/kpu/kenlm.git to /tmp/pip-install-6pgglpio/kenlm_7a1092efcf584bfdb4aebe3347e7d882
  Running command git clone -q git://github.com/kpu/kenlm.git /tmp/pip-install-6pgglpio/kenlm_7a1092efcf584bfdb4aebe3347e7d882
     |████████████████████████████████| 1.7 MB 1.2 MB/s 
     |████████████████████████████████| 28.4 MB 1.8 MB/s 
     |████████████████

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=e10d8250525229105eeef8f3f15c1c4b77c2896ac08c3ef1166054610c7b217f
  Stored in directory: /tmp/pip-ephem-wheel-cache-y61_8r1g/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
/content


#### Restart runtime before you proceed to import MUSS
#### Upload asset_test_dataset.csv from the repo

In [ ]:
from typing import List
from easse.utils.preprocessing import normalize
from easse.utils.text import (
    to_sentences,
    count_words,
    count_syllables_in_sentence,
)


class FKGLScorer:
    "https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests"

    def __init__(self):
        self.nb_words = 0
        self.nb_syllables = 0
        self.nb_sentences = 0

    def add(self, text):
        for sentence in to_sentences(text):
            self.nb_words += count_words(sentence)
            self.nb_syllables += count_syllables_in_sentence(sentence)
            self.nb_sentences += 1

    def score(self):
        # Flesch-Kincaid grade level
        if self.nb_sentences == 0 or self.nb_words == 0:
            return 0
        return max(
            0,
            0.39 * (self.nb_words / self.nb_sentences) + 11.8 * (self.nb_syllables / self.nb_words) - 15.59,
        )


def corpus_fkgl(sentences: List[str], tokenizer: str = "13a"):
    scorer = FKGLScorer()
    for sentence in sentences:
        scorer.add(normalize(sentence, tokenizer=tokenizer))
    return scorer.score()

In [33]:
import pandas as pd
from muss.simplify import simplify_sentences


asset_test = pd.read_csv("asset_test_dataset.csv", encoding="utf-8")
pred_sentences = simplify_sentences(asset_test.text.tolist(), model_name="muss_en_wikilarge_mined")


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


In [34]:
from easse.sari import corpus_sari

corpus_sari(orig_sents=asset_test.text.tolist(),  
            sys_sents=pred_sentences, 
            refs_sents=asset_test[["simp_0","simp_1","simp_2","simp_3","simp_4","simp_5","simp_6","simp_7","simp_8",]].values.T.tolist())


43.61110533872386

In [36]:
corpus_fkgl(pred_sentences)

5.460015407376282

In [40]:
asset_test["preds"] = pred_sentences

In [41]:
turkcorpus_test = pd.read_csv("turkcorpus_test_dataset.csv", encoding="utf-8").dropna()
pred_sentences = simplify_sentences(turkcorpus_test.text.tolist(), model_name="muss_en_wikilarge_mined")


/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


In [42]:
corpus_sari(orig_sents=turkcorpus_test.text.tolist(),  
            sys_sents=pred_sentences, 
            refs_sents=turkcorpus_test[["simp_0","simp_1","simp_2","simp_3","simp_4","simp_5","simp_6","simp_7",]].values.T.tolist())


38.929465037007496

In [43]:
corpus_fkgl(pred_sentences)

5.460015407376282